In [1]:
from pathlib import Path
import sys, os 
import numpy as np
import pandas as pd
import scipy as sci
import xarray as xr

# Settings

In [5]:
# Create output directories if they do not exist

""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({ 'time': new_time })

Kelvin = 273.15       

""" Definitions of kinetic functions """
equation2 = lambda temp: (A_R*np.exp(THETA_A/T_R - THETA_A/(temp + Kelvin)))/(np.exp(THETA_H/T_H - THETA_H/(temp + Kelvin)))
""" Arrhenius equation """
equation3 = lambda temp: B_R*np.exp(THETA_B/T_R - THETA_B/(temp + Kelvin))

experiment_name = 'SODA'
region = 'CelticSea'

out_dir = f'{Path.cwd()}/growth_model_output/{experiment_name}'
if not os.path.exists(out_dir): os.makedirs(out_dir)

# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 2 
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 2000    
# Number of years in the input temperature dataset
years = range(2000, 2005)  	 

# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
lat_coords, lon_coords = slice(47,52), slice(-12,-1)

# Define depth_levels for your growth model output files
depth_levels = slice(0, 600)   # 0-600 meters according to Atlantic cod distribution

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array([47.25, 47.75, 48.25, 48.75, 49.25, 49.75, 50.25, 50.75, 51.25, 51.75], dtype='f')
lon = np.array([
    -11.75, -11.25, -10.75, -10.25, -9.75, -9.25, -8.75, -8.25, -7.75,
    -7.25, -6.75, -6.25, -5.75, -5.25, -4.75, -4.25, -3.75, -3.25, -2.75, -2.25, -1.75,
    -1.25
], dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array([
     -0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 115.,
     135., 160., 190., 230., 280., 340., 410., 490., 580.
], dtype='f')

dt = 1

# Define dimensionality of coords
N_depths, N_lat, N_lon = len(depths), len(lat), len(lon)

input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})


# Functions

In [7]:
default_dim_labels: tuple = ("depth_coord", "latitude", "longitude")
def save2netcdf(data, var_name:str, directory: str, year, age, dimension_labels: tuple=default_dim_labels):
   
    xr.DataArray(
        data, 
        dims = dimension_labels,
        coords = [('latitude', lat), ('longitude', lon)] if 'depth_coord' not in dimension_labels else [('depth_coord', depths), ('latitude', lat), ('longitude', lon)], 
        name = var_name
    ).to_netcdf(f'{directory}_{year}_{var_name}_age-{age}.nc')

# Computation

In [8]:
results = [] # result by year
last_year = initial_year + generation  
while last_year < years[-1]:           
    age, weight = 0, np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
    age=0

    for year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(
            time=f'{year}', 
            depth_coord = depth_levels, 
            latitude = lat_coords, 
            longitude = lon_coords
        )            

        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
            
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan     

        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
        b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

        # mm0 is never used
        if year == initial_year: mm0 = 2 
        else: mm0 = 0

        for mon in np.arange(0,12):
            for dd in np.arange(0,30):
                for ilev in np.arange(0, N_depths):  
                    a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                    b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                    growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :] ** b[ilev, :] - C_AVG )  

                    growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0,0, growth_rates[ilev, :])
                    weight[ilev, :] = weight[ilev, :] * (1. + dt * growth_rates[ilev, :])

        new_year = int(year) + 1
        
        # Reshape data to original shape
        a_3d = a.reshape(N_depths, N_lat, N_lon)
        b_3d = b.reshape(N_depths, N_lat, N_lon)

        growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)

        # # 3D field with asymptotic weight
        weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
        
        # # Calculate maximum asymptotic weight at a given location 
        # # ("W*" in Butzin and Pörtner (2016)) 
        weight_max = np.nanmax(weight_3d, axis = 0)

        # optional
        # results.append({
        #     'weight_3d': weight_3d,
        #     'weight_max': weight_max,
        #     'a_3d': a_3d,
        #     'B_3d': b_3d
        # })
        
        # Save netcdf
        for (data, variable_name) in zip(
            [a_3d, b_3d, growth_rates_3d, weight_3d, weight_max],
            ['a_3d', 'b_3d', 'growth_rates_3d', 'weight_3d', 'weight_max'],
        ):
            save2netcdf(
                data=data, 
                var_name=variable_name, 
                directory=f'{out_dir}/init_initial_year_transient', 
                year=year, 
                age=age, 
                dimension_labels=default_dim_labels if i < 4 else  ('latitude', 'longitude')
            )
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 

/var/folders/j1/k2clnjz530q64v44gw4srhpw0000gn/T/ipykernel_29197/2217209041.py:53: RuntimeWarning: All-NaN slice encountered
  weight_max = np.nanmax(weight_3d, axis = 0)


In [25]:
datasets_weight_3d_max_age_0: dict = {}
for year in range(2000,2003):
    for age in range(2):
        try: datasets_weight_3d_max_age_0[f'{year}-{age}'] = xr.open_dataset(
                f'{out_dir}/init_initial_year_transient_{year}_weight_3d_age-{age}.nc'
        )
        except FileNotFoundError: pass

In [29]:
for year_age, dataset in datasets_weight_3d_max_age_0.items():
    print(year_age)
    # do something with "dataset" 

2000-0
2001-0
2001-1
2002-1
